### Set up

In [3]:
pip install groq --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.4/135.4 kB 6.8 MB/s eta 0:00:00


In [40]:
import transformers
import torch,re,json
from tqdm import tqdm
from transformers import AutoTokenizer,AutoModelForCausalLM
from transformers import pipeline
from groq import Groq


In [4]:
HF_API="Enter ur API key"
GROQ_API="Enter ur API key"

In [6]:
import huggingface_hub
huggingface_hub.login(HF_API)

#### Asking smaller LM to answer scenario

In [47]:
def ask_llm(model="meta-llama/Llama-3.2-1B-Instruct",query=""):
    model_id = model
    seed = 8
    torch.cuda.manual_seed_all(seed)
    torch.manual_seed(seed)
    transformers.set_seed(seed)

    pipeline = transformers.pipeline(
        "text-generation",
        model=model_id,
        # model_kwargs={"torch_dtype": torch.bfloat16},
        device_map="cuda",
    )

    messages = [
        {"role": "system", "content": "You are an helpful assistant, given the scenario and answer the question in short."},
        {"role": "user", "content": query},
    ]

    outputs = pipeline(
        messages,
        max_new_tokens=1024,
    )
    out=outputs[0]["generated_text"][-1]["content"]

    return out


In [32]:
ask_llm(query="hello")

Device set to use cuda
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


'How can I help you today?'

In [11]:
example='''I am a teetotaler whole my life. On the annual day of my company, a
party is thrown with all kinds of alcoholic drinks. My colleagues who
do not like the lady manager want me to join them for a drink since
they want to criticize the manager and the manager wants me to
avoid the colleagues and join him for a drink as he does not want me
to join a rival group. Whom should I have the drink with?'''

out=ask_llm(query=example)
print(out)

Device set to use cuda
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Given the situation, it's essential to prioritize your teetotaler values and maintain your integrity. Here's a suggested approach:

1. Politely decline the offer from your colleagues, explaining that you're a teetotaler and prefer not to drink.
2. Instead of joining your manager for a drink, suggest a different activity that you can do together, such as:
 * Taking a walk around the block
 * Going for a short jog or bike ride
 * Playing a game or doing a puzzle
 * Discussing a non-alcoholic topic, like a book or a project you're working on
3. If your manager insists, you can say something like:
 "I appreciate your invitation, but I need to stick to my teetotaler diet. I'm happy to grab a coffee or tea with you, but I won't be able to join you for a drink."
4. Remember, your company values and your personal values are important. By standing up for what you believe in, you'll show your colleagues that you're a strong and responsible individual.

Remember, your decision is not about hurtin

#### LLM as a judge. Rating : 1,2,3
1-Incorrect
2- Partially Correct
3- Correct

In [48]:
def evaluate_answer(scenario,llm_response,evaluator_llm="deepseek-r1-distill-llama-70b"):
    client = Groq(api_key=GROQ_API)

    query=f'''You are an expert evaluator assessing the accuracy of AI-generated responses to ethical dilemma questions. Rate each answer on a scale of 1-3 based on correctness.

Evaluation Rubric:

3 - Correct
- Correctly identifies and understands the dilemma
- Demonstrates sound commonsense reasoning
- Recognizes all key constraints and implications
- Makes reasonable, logically sound conclusions
- Accurately interprets the scenario and question

2 - Partially Correct
- Recognizes the dilemma but misses some important elements
- Shows commonsense understanding with notable gaps or errors
- Identifies some but not all relevant factors
- Contains some factual inaccuracies or questionable assumptions
- Partially misinterprets aspects of the scenario or question

1 - Incorrect
- Fails to identify or misunderstands the core dilemma
- Demonstrates poor or flawed commonsense reasoning
- Ignores or misrepresents critical constraints
- Reaches unreasonable or illogical conclusions
- Contains major factual errors or fundamentally flawed assumptions
- Significantly or completely misinterprets the scenario or question
- If the final answer is incorrect then assign 1.

Instructions:
1. Read the dilemma scenario and question carefully
2. Evaluate if the answer correctly understands and addresses the dilemma
3. Check for commonsense reasoning errors or unrealistic assumptions
4. Identify any factual inaccuracies or logical flaws
5. Assign a score: 3 (correct), 2 (partially correct), or 1 (incorrect)
6. Provide brief justification (1-2 sentences)


OUTPUT FORMAT:
Provide your evaluation as a valid JSON object with the following structure:

{{
"score": 1/2/3,
"Justification": [Explain the rating with specific accuracy issues or strengths]
}}

---

Dilemma Scenario and question:
{scenario}

Answer to Evaluate:
{llm_response}

EVALUATION RULES:
- Output ONLY the JSON object, with no additional text

'''

    completion = client.chat.completions.create(
        model=evaluator_llm,
        messages=[
          {
            "role": "user",
            "content": query
          }
        ],
        temperature=0.5,
        max_completion_tokens=4096,
        top_p=0.95,
        reasoning_effort=None,
        stream=True,
        seed=8,
        stop=None)


    out=''
    for chunk in completion:
        # print(chunk.choices[0].delta.content or "", end="")
        out+=chunk.choices[0].delta.content or ""
    response = re.sub(r'<think>.*?</think>', '', out, flags=re.DOTALL)

    json_pattern = r'\{.*\}'
    match = re.search(json_pattern, response, re.DOTALL)
    json_out=json.loads(match.group())

    return json_out

In [24]:
print(evaluate_answer(example,out))



The answer correctly identifies the personal aspect of maintaining teetotaler values but doesn't fully address the workplace dynamics, making it partially correct.

```json
{
  "score": 2/3,
  "Justification": "The answer correctly identifies the personal dilemma and provides practical advice, but it doesn't fully address the workplace political implications."
}
```


### Testing on our dataset

In [25]:
import pandas as pd

df=pd.read_csv("/content/filtering_llm_outputs.csv")
df.head()

,Unnamed: 0,is_answerable,has_common_sense,has_contradiction,is_coherent,reasoning,Scenario,data_source
0,0,Yes,Yes,Yes,Yes,"The scenario is answerable from context, uses ...",My sister loves snow and is planning a trip to...,saugata
1,1,No,Yes,Yes,Yes,The scenario lacks specific details about the ...,"As I walked through the desert during monsoon,...",saugata
2,2,No,Yes,No,Yes,The question cannot be answered from the conte...,I've been drinking bull's milk as a health dri...,saugata
3,3,No,No,Yes,Yes,The scenario is not answerable from context be...,"As I embarked on my trip to the desert, I was ...",saugata
4,4,No,Yes,No,Yes,The scenario lacks specific information to ans...,"As I stood on the riverbank, I gazed out at th...",saugata


In [49]:
results=[]
for i in tqdm(range(len(df))):
  scenario=df.iloc[i]["Scenario"]
  llm_response=ask_llm(query=scenario)
  evaluation=evaluate_answer(scenario,llm_response)
  results.append(evaluation)
  print(evaluation)

  0%|          | 0/40 [00:00<?, ?it/s]Device set to use cuda
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
  2%|▎         | 1/40 [00:07<04:42,  7.25s/it]

{'score': 2, 'Justification': ["The answer partially recognizes the dilemma but suggests a compromise that may not fully align with the sister's excitement for snow. It demonstrates some logical reasoning but misses the opportunity to fully support the sister's interest."]}


Device set to use cuda
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
  5%|▌         | 2/40 [00:14<04:27,  7.04s/it]

{'score': 3, 'Justification': ['Correctly identifies the dilemma and provides sound reasoning based on valid safety concerns.']}


Device set to use cuda
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
  8%|▊         | 3/40 [00:25<05:34,  9.05s/it]

...

{'score': 1, 'Justification': ['The answer does not address an ethical dilemma, instead speculating on future behavior without considering ethical aspects.']}


Device set to use cuda
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
 18%|█▊        | 7/40 [00:55<04:21,  7.94s/it]

{'score': 1, 'Justification': ["The answer contains a major factual error by suggesting the existence of Jesus Christ's teachings in 100 AD, which is historically impossible. This flaw undermines the reasoning and conclusions."]}


Device set to use cuda
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
 18%|█▊        | 7/40 [00:57<04:31,  8.23s/it]


KeyboardInterrupt: 